In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm import tqdm
import os 

import matplotlib.pylab as pl

In [2]:
import axisrules as axis

In [3]:
path_dataset = "../data/" 

list_datasets = []

for folder in os.listdir(path_dataset):
    for dataset in os.listdir(path_dataset+folder):
        candidates = path_dataset+folder+"/candidates.csv"
        # Test isdir 
        if os.path.isdir(path_dataset+folder+"/"+dataset):
            path_weights=  path_dataset+folder+"/"+dataset+"/weights.csv" 
            # Test if there is weight file:
            isweight = False
            if os.path.isfile(path_weights):
                isweight = True
            for file in os.listdir(path_dataset+folder+"/"+dataset+"/votes"):
                if file.endswith(".csv"):
                    list_datasets.append({"votes":path_dataset+folder+"/"+dataset+"/votes/"+file, 
                                            "weights":path_weights,
                                            "isweight":isweight,
                                            "candidates": candidates,
                                            "year": folder,
                                            "dataset": dataset,
                                            "file":file})




In [4]:
# keep Online of France 2017:

data = list_datasets[8]

In [5]:
data

{'votes': '../data/France 2017/Online/votes/approval.csv',
 'weights': '../data/France 2017/Online/weights.csv',
 'isweight': True,
 'candidates': '../data/France 2017/candidates.csv',
 'year': 'France 2017',
 'dataset': 'Online',
 'file': 'approval.csv'}

In [6]:
df = pd.read_csv(data["votes"], index_col=0)
candidates = list(df.columns)
votes = df.to_numpy()
df_weights = pd.read_csv(data["weights"], index_col=0)
weights = []
for i in range(len(df.index)):
    ind = df.index[i]
    if ind in df_weights.index:
        weights.append(df_weights.loc[ind]["approval"])
    else:
        weights.append(0)
weights = np.array(weights)


In [8]:
a1 = axis.VoterDeletion(votes, weights).bruteforce()

 66%|██████▌   | 239416/362880 [00:21<00:04, 29112.35it/s]

In [7]:
a2 = axis.KScoringRule(2, axis.VoterDeletion, votes, weights).bruteforce()

  0%|          | 1482/362880 [01:02<4:12:57, 23.81it/s]


KeyboardInterrupt: 

In [8]:
a2

[[2, 1, 7, 3, 4, 8, 5, 9, 10, 0, 6], [2, 1, 7, 3, 4, 8, 5, 9, 10, 0, 6]]

In [7]:
a1 = axis.VoterDeletion(votes, weights).bruteforce()

100%|██████████| 362880/362880 [00:25<00:00, 14007.15it/s]


In [24]:
id_not_covered = []
v = axis.VoterDeletion(votes, weights)
for i in range(len(votes)):
    if v.get_ballot_score(a1[0][0], votes[i], np.sum(votes[i])) !=0:
        id_not_covered.append(i)


In [25]:
a2 = axis.VoterDeletion(votes[id_not_covered], weights[id_not_covered]).bruteforce()

100%|██████████| 362880/362880 [51:00<00:00, 118.56it/s] 


In [33]:
axis.print_axis(a1[0][0], candidates)

Jacques Cheminade < François Asselineau < Marine Le Pen < Nicolas Dupont-Aignan < François Fillon < Emmanuel Macron < Benoît Hamon < Jean-Luc Mélenchon < Philippe Poutou < Nathalie Arthaud < Jean Lassalle


In [34]:
axis.print_axis(a2[0][0], candidates)

Jacques Cheminade < François Asselineau < Nicolas Dupont-Aignan < Jean-Luc Mélenchon < Marine Le Pen < François Fillon < Jean Lassalle < Emmanuel Macron < Benoît Hamon < Philippe Poutou < Nathalie Arthaud
